In [ ]:
import onnxruntime as ort
import torch
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
from util import non_max_suppression
import os
from onnxruntime.quantization import quantize_dynamic, QuantType

In [2]:


# Device configuration
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def preprocess_image(image_path, input_size=640):
    """Preprocess image to match training pipeline in Yolo_Dataset."""
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load image: {image_path}")
    
    # Get original shape
    h, w = image.shape[:2]
    
    # Scale ratio (new / old)
    r = min(input_size / h, input_size / w)
    
    # Resize
    pad = (int(w * r), int(h * r))
    if (h, w) != pad[::-1]:
        image = cv2.resize(image, dsize=pad, interpolation=cv2.INTER_LINEAR)
    
    # Compute padding
    top, bottom = int((input_size - pad[1]) / 2), int((input_size - pad[1]) / 2 + (input_size - pad[1]) % 2)
    left, right = int((input_size - pad[0]) / 2), int((input_size - pad[0]) / 2 + (input_size - pad[0]) % 2)
    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
    
    # Convert HWC to CHW, BGR to RGB, normalize
    image = image.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    image = np.ascontiguousarray(image) / 255.0  # Normalize to [0, 1]
    
    # Convert to float32 for ONNX
    image = image.astype(np.float32)
    
    return image, (r, r), (left, top), (h, w)

def draw_detections(image, detections, class_names=None):
    """Draw bounding boxes and labels on the image."""
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        label = f"{int(cls)} {conf:.2f}" if class_names is None else f"{class_names[int(cls)]} {conf:.2f}"
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return image

def onnx_inference(session, image_paths, input_size=640, conf_thres=0.25, iou_thres=0.65, output_dir="output_onnx"):
    """Run inference on a list of images using ONNX model and save results."""
    os.makedirs(output_dir, exist_ok=True)
    
    for image_path in tqdm(image_paths, desc="ONNX Inference"):
        # Preprocess image
        image_np, ratio, pad, orig_shape = preprocess_image(image_path, input_size)
        image_np = image_np[np.newaxis, ...]  # Add batch dimension (1, 3, 640, 640)
        
        # Run ONNX inference
        input_name = session.get_inputs()[0].name
        output_name = session.get_outputs()[0].name
        outputs = session.run([output_name], {input_name: image_np})[0]
        
        # Convert ONNX output to PyTorch tensor for NMS
        outputs = torch.from_numpy(outputs).to(DEVICE)
        
        # Apply non-maximum suppression
        detections = non_max_suppression(outputs, confidence_threshold=conf_thres, iou_threshold=iou_thres)
        
        # Process detections
        detections = detections[0]  # First batch
        if detections.shape[0] > 0:
            # Rescale boxes to original image size
            detections[:, :4] = detections[:, :4].clone()
            detections[:, [0, 2]] = (detections[:, [0, 2]] - pad[0]) / ratio[0]  # x1, x2
            detections[:, [1, 3]] = (detections[:, [1, 3]] - pad[1]) / ratio[1]  # y1, y2
        
        # Load original image for visualization
        orig_image = cv2.imread(image_path)
        if detections.shape[0] > 0:
            orig_image = draw_detections(orig_image, detections)
        else:
            print(f"No detections for {image_path}")
        
        # Save output
        output_path = os.path.join(output_dir, os.path.basename(image_path))
        cv2.imwrite(output_path, orig_image)
    
    print(f"Results saved to {output_dir}")




In [3]:
onnx_model_path = "/mnt/DATA/gits/yolov11_scratch/weights/my_model.onnx"

# Initialize ONNX runtime session
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] if torch.cuda.is_available() else ['CPUExecutionProvider']
try:
    session = ort.InferenceSession(onnx_model_path, providers=providers)
    print(f"Loaded ONNX model from {onnx_model_path}")
except Exception as e:
    print(f"Failed to load ONNX model: {e}")


# Input images (e.g., validation set or test images)
image_paths = glob('/mnt/DATA/DATASETS/data/dlpj/slp/val/images/*.jpg')  # Adjust path as needed

# Inference parameters
input_size = 640
conf_thres = 0.1  # Lowered to allow more detections
iou_thres = 0.5   # Lowered to reduce NMS filtering
output_dir = "output_onnx/inference_results"

# Run inference
onnx_inference(session, image_paths, input_size, conf_thres, iou_thres, output_dir)

Loaded ONNX model from /mnt/DATA/gits/yolov11_scratch/weights/my_model.onnx


ONNX Inference:   0%|          | 1/721 [00:01<19:16,  1.61s/it]

No detections for /mnt/DATA/DATASETS/data/dlpj/slp/val/images/DJI_20240114104931_0053_T_-_JPG.rf.6b097ea0b5143cc9b32f7166ce5d8247.jpg


ONNX Inference:   1%|          | 9/721 [00:02<01:35,  7.44it/s]

No detections for /mnt/DATA/DATASETS/data/dlpj/slp/val/images/DJI_20240114112233_0038_T_-_JPG.rf.edc7f15cae23acb0f83339c86ad941b6.jpg
No detections for /mnt/DATA/DATASETS/data/dlpj/slp/val/images/DJI_20240121114851_0016_T_-_JPG.rf.e5744025d74d1e294551ef054eb3ec36.jpg


...

No detections for /mnt/DATA/DATASETS/data/dlpj/slp/val/images/DJI_20240114104815_0009_T_-_JPG.rf.3ad598ff3f0afada6f9eebe72a9d8b0b.jpg
No detections for /mnt/DATA/DATASETS/data/dlpj/slp/val/images/DJI_20240114103752_0082_T_-_JPG.rf.dc1ea173a2c36f81a1e009eefcbcf1ee.jpg


ONNX Inference:  10%|▉         | 69/721 [00:04<00:22, 28.81it/s]

No detections for /mnt/DATA/DATASETS/data/dlpj/slp/val/images/DJI_20240603133248_0074_T_-_jpeg.rf.f255728c72c239926cdf25ed7684b116.jpg
No detections for /mnt/DATA/DATASETS/data/dlpj/slp/val/images/DJI_20240121120733_0069_T_-_JPG.rf.7aeffb59dd085478bea54902fc212c9b.jpg


ONNX Inference:  10%|▉         | 71/721 [00:04<00:39, 16.33it/s]


KeyboardInterrupt: 